In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import numpy as np
from scipy import stats
import pandas as pd
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from statsmodels.formula.api import ols

X_train = pd.read_csv('bakeoff_data/Xtrain.csv')
y_train = pd.read_csv('bakeoff_data/ytrain.csv')
X_test = pd.read_csv('bakeoff_data/Xtest.csv')


In [2]:
X_test['sqft_basement'] = X_test['sqft_basement'].apply(lambda x: np.nan if "?" in x else x)

In [3]:
X_train['sqft_basement'] = X_train['sqft_basement'].apply(lambda x: np.nan if "?" in x else x)

In [4]:
X_test_no_na = X_test.fillna({'waterfront': 0, 'view': 0, 'yr_renovated': 0, 'sqft_basement': 0})
X_train_no_na = X_train.fillna({'waterfront': 0, 'view': 0, 'yr_renovated': 0, 'sqft_basement': 0})

In [5]:
X_train_no_na['sqft_basement'] = X_train_no_na['sqft_basement'].astype(float)
X_test_no_na['sqft_basement'] = X_test_no_na['sqft_basement'].astype(float)

In [6]:
X_train_drop = X_train_no_na.drop(['date', 'long', 'lat'], axis = 1)
X_test_drop = X_test_no_na.drop(['date', 'long', 'lat'], axis = 1)

In [7]:
X_test_drop['sqft_x_grade'] = X_test_drop['sqft_living'] * X_test_drop['grade']

In [8]:
X_train_drop['sqft_x_grade'] = X_train_drop['sqft_living'] * X_train_drop['grade']

In [9]:
X_train_drop.isna().sum()

bedrooms         0
bathrooms        0
sqft_living      0
sqft_lot         0
floors           0
waterfront       0
view             0
condition        0
grade            0
sqft_above       0
sqft_basement    0
yr_built         0
yr_renovated     0
zipcode          0
sqft_living15    0
sqft_lot15       0
sqft_x_grade     0
dtype: int64

In [10]:
X_test_drop.isna().sum()

bedrooms         0
bathrooms        0
sqft_living      0
sqft_lot         0
floors           0
waterfront       0
view             0
condition        0
grade            0
sqft_above       0
sqft_basement    0
yr_built         0
yr_renovated     0
zipcode          0
sqft_living15    0
sqft_lot15       0
sqft_x_grade     0
dtype: int64

In [11]:
from sklearn.preprocessing import OneHotEncoder
zipcode = X_train_drop[['zipcode']]
ohe = OneHotEncoder(categories='auto', sparse=False, handle_unknown='ignore')
ohe.fit(zipcode)
ohe.categories_
zip_encoded = ohe.transform(zipcode)

zip_encoded = pd.DataFrame(

    zip_encoded,

    columns=ohe.categories_[0],
 
    index=X_train_drop.index
)

zip_encoded = zip_encoded.drop(98001, axis=1)
zip_encoded

,98002,98003,98004,98005,98006,98007,98008,98010,98011,98014,...,98146,98148,98155,98166,98168,98177,98178,98188,98198,98199
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16192,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16193,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
16194,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16195,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [12]:
X_train_drop = X_train_drop.drop('zipcode', axis = 1)

In [13]:
X_train_drop = pd.concat([X_train_drop, zip_encoded], axis = 1)

In [14]:
zipcode_test = X_test_drop[['zipcode']]
ohe = OneHotEncoder(categories='auto', sparse=False, handle_unknown='ignore')
ohe.fit(zipcode)
ohe.categories_
zip_encoded_test = ohe.transform(zipcode_test)

zip_encoded_test = pd.DataFrame(

    zip_encoded_test,

    columns=ohe.categories_[0],
 
    index=X_test_drop.index
)
zip_encoded_test = zip_encoded_test.drop(98001, axis=1)

zip_encoded_test

,98002,98003,98004,98005,98006,98007,98008,98010,98011,98014,...,98146,98148,98155,98166,98168,98177,98178,98188,98198,98199
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5395,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5396,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5397,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5398,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
X_test_drop = X_test_drop.drop('zipcode', axis = 1)

In [16]:
X_test_drop = pd.concat([X_test_drop, zip_encoded_test], axis = 1)

In [17]:
X_test_drop

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,...,98146,98148,98155,98166,98168,98177,98178,98188,98198,98199
0,3,0.75,850,8573,1.0,0.0,0.0,3,6,600,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3,1.00,1510,6083,1.0,0.0,0.0,4,6,860,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4,2.25,1790,42000,1.0,0.0,0.0,3,7,1170,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2,1.50,1140,2500,1.0,0.0,1.0,3,7,630,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3,1.00,1500,3920,1.0,0.0,0.0,3,7,1000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5395,5,1.00,1170,6757,1.0,0.0,0.0,4,6,800,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5396,3,1.75,1670,5100,1.0,0.0,2.0,5,7,990,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5397,4,2.25,3260,4640,2.0,0.0,0.0,5,9,2360,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5398,4,5.00,5820,13906,2.0,0.0,0.0,3,11,3750,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


hi max and michelle, 

sorry for the late response, just getting into the survey. I’m know seeing max ## Regression


In [18]:
from sklearn.model_selection import train_test_split
lr_one = LinearRegression()
X_t, X_val, y_t, y_val = train_test_split(X_train_drop, y_train, random_state=42, test_size=.2)

In [19]:
lr_one.fit(X_t,y_t)
lr_one.score(X_t,y_t)

0.8470976436062453

In [20]:
lr_one.score(X_val, y_val)

0.8168135635313368

In [21]:
from sklearn.linear_model import LinearRegression

In [22]:
lr = LinearRegression()
lr.fit(X_train_drop, y_train)
lr.score(X_train_drop,y_train)

0.8426028457813262

In [23]:
y_hat = lr.predict(X_test_drop)
y_hat.shape

(5400, 1)

## Log Transform

In [24]:
lr_log = LinearRegression()
y_t_log = np.log(y_t)
y_val_log = np.log(y_val)
lr_log.fit(X_t, y_t_log)
lr_log.score(X_t,y_t_log)

0.8818928150086566

In [25]:
lr_log.score(X_val, y_val_log)

0.8693348178730417

## Final

In [26]:
y_hat_log = lr_log.predict(X_test_drop)

In [27]:
y_hat = np.e ** y_hat_log

In [28]:
import numpy as np
y_test_fake = np.full((5400,1), 0)

In [31]:
from sklearn.metrics import r2_score

# fake predictions using the mean of y_train.
your_y_hat_predictions = np.full((5400,1), np.mean(y_train))

r2_score(y_hat, y_test_fake)

-2.219314182066588

In [32]:
np.savetxt('kyle_kevin_vac_pandas.csv', y_hat, delimiter=',')